In [1]:
    import os
    import time
    import datetime
    import pandas as pd
    from ftplib import FTP

In [5]:
    start_time = time.time()

    df = pd.read_json(os.path.join("data_raw","martinus_sledovat.json"))

    dk = pd.read_csv(os.path.join("data","databazeknih-hodnoceni.csv"))
    gr = pd.read_csv(os.path.join("data","goodreads-hodnoceni.csv"))

    df['isbn'] = df['M_isbn'].astype(str).apply(lambda x: x.replace(".0",""))
    dk['isbn'] = dk['DK_isbn'].astype(str).apply(lambda x: x.replace(".0",""))

    def hezky_titul(row):
        try:
            autorstvo = df[df['isbn'] == row['isbn']].explode('M_autorstvo')['M_autorstvo'].drop_duplicates().to_list()[0]
            return f"""{autorstvo}: {row['DK_titul']}"""
        except:
            return f"""?: {row['DK_titul']}"""
        
    dk['hezky_titul'] = dk.apply(lambda row: hezky_titul(row), axis=1)
    dk['skore'] = dk['DK_Chystám_se_číst'] + dk['DK_Přečtených']

    month_ago = pd.to_datetime(gr['GR_date'].max()) - datetime.timedelta(days=29)
    month_ago = month_ago.strftime('%Y-%m-%d')


In [13]:
dk['hezky_titul']

0                   Zdeněk Řehák: Boží potměšilost
1              Roman Cílek: Oprátka za osm mrtvých
2                             Karel Štorkán: Golem
3         Zuzana Dostálová: Proč všichni odcházejí
4                           Artur Janoušek: Sochař
                           ...                    
36415    Kateřina Přidalová: Toto není jenom kniha
36416             Eliška Beranová: Sbírání borůvek
36417       Jaromír Kubů: Noci a dny na Karlštejně
36418              Anna Jonáková: Havlíčku, Havle!
36419                    Vojtěch Kessler: 3.7.1866
Name: hezky_titul, Length: 36420, dtype: object

In [15]:
    dk_stare = dk[dk['DK_date'] < month_ago]['hezky_titul'].drop_duplicates().to_list()
    dk_nove = dk[~dk['hezky_titul'].isin(dk_stare)]['hezky_titul'].drop_duplicates().to_list()
    gr_stare = gr[gr['GR_date'] < month_ago]['GR_title'].drop_duplicates().to_list()
    gr_nove = gr[~gr['GR_title'].isin(gr_stare)]['GR_title'].drop_duplicates().to_list()

In [17]:
dk_nove

['Ladislav Jakl: IVK k nedávným evropským volbám',
 'Hana Krutílková: Disciplinované buřičky: Ženy v politických stranách na Moravě do roku 1914',
 'Milena Jesenská: Milena Jesenská: Dopisy a dokumenty',
 'Anne Franková: Zadní dům: Deník',
 'Aleš Pejchal: Zpráva o stavu Evropského soudu pro lidská práva',
 'Monika Nevolová: Přestaň nudit, začni bavit',
 'Michal Řoutil: Křesťanská Golgota v Malé Asii (1914–1923): Svědectví, vzpomínky, dokumety',
 'Michal Dlouhý: Případy z četnických zápisníků',
 'Jiří Pátek: Dagmar Hochová',
 'Monika Klemšová: Na paddleboardu za snem',
 'Eva Michorová: Zpověď stárnoucí trosky, která má psa',
 'Dora Kaprálová: Zimní kniha o lásce',
 'Veronika Martinková: Mrtvá budoucnost',
 'Vlastimil Vondruška: Nevěsta Kristova',
 'František Skácel: Jízdní kolo lidé, historie, technika',
 'Aleš Dvořák: Zeptej se vědce: Neexistují blbé otázky',
 'Jan  Novotný: Básníci a kosmonauté',
 'Jiří Anger: Digitální Kříženecký: Nový život prvních českých filmů',
 'Miloslav Nevrlý:

In [19]:
gr_nove

['Skutečné zločiny v Brně a okolí by Ivana Žáková',
 'Lež má krátké nohy by Karol Efraim Sidon',
 '#Nezapomeň. Příběhy Paměti národa by Irena Tatíčková',
 'Květ zla (Nevěsta na útěku, #3) by Alexander Stainforth',
 'České sklo by Marek Toman',
 'Nevěsta na útěku (Nevěsta na útěku, #1) by Alexander Stainforth',
 'Jak se líčit v nemoci by Iryna Zahladko',
 'Řeč mrtvejch / Erzherz / Nebožky by Radek Fridrich',
 'Past na ženicha (Nevěsta na útěku, #2) by Alexander Stainforth',
 'Zeptej se vědce: Neexistují blbé otázky by Aleš Dvořák',
 'Endometrióza v otázkách a odpovědích by Jan Drahoňovský',
 'Exploring the Benefits of Nature on Mental Health by Oheta Sophia',
 'Prý musím projít březovým lesem by Štěpánka Borská',
 'Metakunsthistorie by Jakub Stejskal',
 'Průvodce králičí norou by Jan Tománek',
 'Mezera mezi zdí a parketama, část první by Jaromír František Fumas Palme',
 'Mediátorka: Každý má svou pravdu by Hana Lenghartová',
 '…a přeskočila jiskra by Halina Pawlowská',
 'Indie: Královna

In [21]:
    dk_mesic_celkem = pd.Series(dk.groupby("hezky_titul")['skore'].max() - dk[dk['DK_date'] > month_ago].groupby("hezky_titul")['skore'].min()).sort_values(ascending=False)
    dk_mesic_celkem = dk_mesic_celkem[dk_mesic_celkem >= 30]
    dk_mesic_novinky = pd.Series(dk[~dk['hezky_titul'].isin(dk_stare)].groupby("hezky_titul")['skore'].max() - dk[(~dk['hezky_titul'].isin(dk_stare)) & (dk['DK_date'] > month_ago)].groupby("hezky_titul")['skore'].min()).sort_values(ascending=False)
    dk_mesic_novinky = dk_mesic_novinky[dk_mesic_novinky >= 10]

In [23]:
df.columns

Index(['index', 'M_titul', 'M_autorstvo', 'M_isbn', 'isbn'], dtype='object')

In [45]:
    def nakladatelstvi(isbn):
        try:
            return df[df['isbn'].str == str(isbn)]['M_nakladatel'].drop_duplicates().to_list()[0]
        except:
            return None

    dk['nakladatel'] = dk['isbn'].apply(lambda x: nakladatelstvi(x))

In [47]:
dk['nakladatel']

0        None
1        None
2        None
3        None
4        None
         ... 
36415    None
36416    None
36417    None
36418    None
36419    None
Name: nakladatel, Length: 36420, dtype: object

In [43]:
dk[dk['DK_date'] > month_ago]

,DK_isbn,DK_date,DK_titul,DK_rating,DK_ratings_count,DK_tags,DK_Právě_čtených,DK_Přečtených,DK_Čtenářské_výzvě,DK_Knihotéce,DK_Chystám_se_číst,DK_Chci_si_koupit,DK_dalších_seznamech,DK_Doporučených,isbn,hezky_titul,skore,nakladatel
29026,9.788088e+12,2024-08-19 03:46:12,Jindřich Chalupecký: Texty a kontexty kritika ...,100.0,1.0,['Umění'],NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,9788088366478,Tomáš Pospiszyl: Jindřich Chalupecký: Texty a ...,NaN,None
29027,9.788076e+12,2024-08-19 03:46:13,Strach a úzkost a jak na ně,76.0,18.0,"['Literatura česká', 'Literatura naučná', 'Psy...",NaN,24.0,3.0,9.0,32.0,16.0,1.0,NaN,9788076018587,?: Strach a úzkost a jak na ně,56.0,None
29028,9.788088e+12,2024-08-19 03:46:14,Skica jako prvotní vizualizace myšlenky: Kdy n...,100.0,1.0,['Umění'],NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,9788088378310,David Böhm: Skica jako prvotní vizualizace myš...,2.0,None
29029,9.788020e+12,2024-08-19 03:46:16,Richard Wagner a jeho Židovství v hudbě,80.0,2.0,"['Literatura naučná', 'Filozofie', 'Historie']",NaN,1.0,NaN,NaN,1.0,2.0,NaN,NaN,9788020034663,Pavel Sládek: Richard Wagner a jeho Židovství ...,2.0,None
29030,9.788081e+12,2024-08-19 03:46:17,V zajatí narcizmu,85.0,4.0,"['Literatura naučná', 'Psychologie a pedagogika']",NaN,5.0,1.0,NaN,3.0,3.0,NaN,1.0,9788080905545,Robert Tomšik: V zajatí narcizmu,8.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36415,9.788088e+12,2024-09-16 11:37:51,Toto není jenom kniha,93.0,3.0,"['Literatura naučná', 'Umění']",NaN,4.0,NaN,1.0,11.0,6.0,NaN,NaN,9788088308898,Kateřina Přidalová: Toto není jenom kniha,15.0,None
36416,9.788075e+12,2024-09-16 11:37:51,Sbírání borůvek,90.0,2.0,"['Novely', 'Literatura česká']",NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,9788075304476,Eliška Beranová: Sbírání borůvek,3.0,None
36417,9.788020e+12,2024-09-16 11:37:52,Noci a dny na Karlštejně,85.0,8.0,"['Literatura česká', 'Architektura', 'Cestopis...",1.0,13.0,4.0,1.0,10.0,2.0,NaN,NaN,9788020462763,Jaromír Kubů: Noci a dny na Karlštejně,23.0,None
36418,9.788087e+12,2024-09-16 11:37:53,"Havlíčku, Havle!",80.0,1.0,"['Literatura naučná', 'Biografie a memoáry', '...",NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,9788087302453,"Anna Jonáková: Havlíčku, Havle!",2.0,None


In [28]:
    pribytek = pd.DataFrame(dk[dk['DK_date'] > month_ago].groupby(['hezky_titul','nakladatel'])['skore'].max() - dk[dk['DK_date'] > month_ago].groupby(['hezky_titul','nakladatel'])['skore'].min())

In [29]:
pribytek

,,skore
hezky_titul,nakladatel,
